## Семинар 13
# Тема: Кластеризация

_Кластеризация_ относится к методам обучения без учителя. 

Задача кластеризации состоит в разбиении множества исходных объектов на подмножества близких по свойствам объектов. Понятие _близости_ формализуется при помощи задания _метрики_, или расстояния между объектами $\rho\left(x_i,x_j\right)$. 

### Метод k-means.
Познакомимся с одним из наиболее известных алгоритмов кластеризации: _k-means_ или _k средних_. 

Алгоритм метода выглядит следующим образом:

1. Случайным образом выбираются k точек $x_k \in X$, в дальнейшем будем называть их центрами кластеров (центроидами);

2. Для каждой точки $x_i \in X$ вычисляется $\rho\left(x_i,x_k\right)$, после чего ей ставится в соответствие  ближайший к ней центр кластера. Совокупность точек, отнесённых к центру $x_k$ называется кластером;

3. Вычисляются средние взвешенные точек (центры масс), принадлежащих к определённому кластеру. Эти значения становятся новыми центрами кластеров;

4. Шаги 2 и 3 повторяются до тех пор, пока пересчёт центров кластеров будет существенно менять местоположение кластеров. Как только высчитанные центры кластеров совпадут с предыдущими, алгоритм будет окончен.

Следует иметь в виду, что этот алгоритм хорошо работает только для нормальных выборок, причем кластеры не должны существенно перекрываться.

Недостатком алгоритма является то, что число кластеров в нем - гиперпараметр, т.е. должно быть задано изначально.


### Метод DBSCAN.

Познакомиться с методом машинного обучения DBSCAN, осуществляющим группировку наблюдений в кластеры высокой плотности.

DBSCAN имеет три основных устанавливаемых параметра:

♦ eps — максимальное расстояние от наблюдения, чтобы считать другое наблюдение его соседом;

♦ min sampies — минимальное число наблюдений, находящихся на расстоянии менее eps от наблюдения, для того чтобы его можно было считать ключевым наблюдением;

♦ metric— метрический показатель расстояния, используемый параметром eps, 
например minkowski или euclidean (обратите внимание, что если используется
расстояние Минковского, то может быть использован параметр р для установки
мощности метрического показателя Минковского).

Наблюдения подразделяются на три типа точек:

1. Центральная точка (точка данных является центральной, если круг радиуса eps вокруг нее содержит не менее min_sampies точек)

2. Граничная точка (количество точек меньше min_sampies, но есть хотя бы одна центральная)

3. Шум (нет ни одной центральной точки в круге радиуса eps вокруг нее)

Алгоритм хорошо работает с кластерами произвольной формы, число кластеров определяется в ходе работы алгоритма, а не задаётся, как гиперпараметр. Хуже работает на данных с разной плотностью.

Silhouette Score — это метрика для оценки качества результатов кластеризации. Она измеряет, насколько точка данных похожа на собственный кластер по сравнению с другими кластерами. Оценка варьируется от –1 до 1, где более высокое значение указывает на лучшую производительность кластеризации. Значение, близкое к 1, предполагает, что точки данных хорошо кластеризованы и правильно разделены, а значение, близкое к -1, указывает на то, что точки данных могли быть отнесены к неправильным кластерам. В коде Silhouette Score рассчитывается с помощью функции silhouette_score().

Загрузим необходимые библиотеки:

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import datasets

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

# Метод k-means.

### 1. Сформируйте данные при помощи make_blobs(n_samples=1000, centers=5, random_state=10). Выведите полученные данные в виде датафрейма, дав названия колонкам X1 и  X2. Визуализируйте их при помощи диаграммы рассеяния.

### 2. Масштабируйте данные при помощи MinMaxScaler.

### 3. Создайте экземпляр класса k-средних с пятью кластерами, зафиксировав random_state = 42. Обучите эту модель на масштабированных данных. Выведите метки кластеров при помощи атрибута .labels_ и при помощи метода .predict(). Убедитесь, что результат будет одинаковый.

### 4. Выведите центры кластеров с помощью атрибута .cluster_centers_. Ремасштабируйте центры кластеров при помощи метода .inverse_transform().

### 5. Визуализируйте полученные кластеры, подкрасив их разными цветами. Также на этом рисунке укажите центры кластеров, пометив их красными ромбами.

### 6. Выведите сумму квадратов расстояний образцов до ближайшего центра кластера при помощи атрибута .inertia_

### 7. Осуществите проверку правильности выбора гиперпараметра модели k-средних "количество кластеров" (n_clusters=5) методом локтя. Для этого постройте график зависимости количества кластеров от значения inertia_. Правильное количество кластеров будет тo, где построенная кривая имеет наиболее сильный излом ("локоть"), то есть то, после которого значение inertia_ перестаёт существенно снижаться.

### 8. Оцените качество кластеризации при помощи метрики Silhouette Score.

# Метод DBSCAN.

### 1. Загрузите встроенный датасет iris. В качестве рассматриваемых данных, возьмите второй и третий столбцы датасета. Выведите данные в виде датафрейма c названиями колонок. 

### 2. Визуализируйте данные на диаграмме рассеяния.

### 3. Стандартизируйте данные при помощи StandardScaler.

### 4. Создайте класс DBSCAN(eps=0.65, min_samples=8,n_jobs=-1). Обучите на стандартизированных данных. Выведите метки кластеров при помощи атрибута .labels_.

### 5. Выведите получившееся в результате работы алгоритма, количество кластеров.

### 6. Выведите количество наблюдений, отнесённых к шумам. Для шумов метка кластеров равна -1.

### 7. Визуализируйте полученные кластеры и шумы, подкрасив их разными цветами. 

### 8. Оцените качество кластеризации при помощи метрики Silhouette Score.

### Задания самостоятельного для выполнения:

### Метод k-means.
1. Загрузите прилагающийся датасет Mall_Customers.csv.
2. Оставьте в нём только два столбца Annual Income и Spending Score.
3. Масштабируйте данные при помощи StandardScaler.
4. Визуализируйте данные при помощи диаграммы рассеяния и сделайте предположение о количестве кластеров. 
5. Определите наилучшее число кластеров с помощью метода локтя.
6. Обучите модель k-средних с лучшим числом кластеров и предскажите метки кластеров для каждого значения из датасета.
7. Визуализируйте полученные кластеры разными цветами с изображением центров кластеров.
8. Оцените эффективность кластеризации с помощью метрики Silhouette Score.

### Метод DBSCAN.
1. Загрузите встроенный датасет iris. Выведите данные в виде датафрейма c названиями колонок. 
2. Стандартизируйте данные при помощи StandardScaler.
3. Создайте класс DBSCAN(n_jobs=-1). Обучите на стандартизированных данных. Выведите метки кластеров при помощи атрибута .labels_.
4. Выведите получившееся в результате работы алгоритма, количество кластеров.
5. Выведите количество наблюдений, отнесённых к шумам. 
6. Оцените качество кластеризации при помощи метрики Silhouette Score.
7. Создайте класс DBSCAN(eps=0.65, min_samples=8,n_jobs=-1). Обучите на стандартизированных данных. Выведите метки кластеров при помощи атрибута .labels_. Оцените качество кластеризации при помощи метрики Silhouette Score и сделайте вывод о том, улучшилась ли работа алгоритма при изменении гиперпараметров модели. 
8. Повторите задание 7, используя Pipeline.